# Einsendeaufgabe 2: Springende Bälle und Pendel (100 Punkte)

## Springende Bälle (60 Punkte)
In dieser Übung wollen wir einen NumPy Code entwickeln, den wir in der nächsten Übung optimieren. 

Wir wollen Bälle simulieren, die in einer Box herumspringen. Die Box sei von $x=[0,xmax]$ und $y=[0,ymax]$ definiert. Jede Kugel hat eine Koordinate $c=[x,y]$ und eine Geschwindigkeit $v=[v_x,v_y]$. In jedem Zeitschritt $\Delta t$ (dt) legt die Kugel eine gewisse Strecke zurück.

Es gilt nach den Regeln der Physik

$c[t+\Delta t]=c[t]+\Delta t*v(t)$. 

 
Zusätzlich wirkt in y-Richtung die Schwerkraft, welche die Geschwindigkeit verändert:
 
$v[t+\Delta t]=v[t]+\Delta t*g$. 


Wenn ein Ball an die Wand stößt, (also z.B. $c_0<=0$ oder $c_0>=xmax$ wird er reflektiert, das heißt in erster Näherung $v_0 = -1 * v_0$ für die Geschwindigkeit in dieser Richtung). Damit der Ball in diesem Fall nicht außerhalb der Box landet, müssen Sie berechnen, wie der Ball nun fliegt.

Der Ball stoppt nicht an der Wand, wenn er dort ankommt, sondern  fliegt noch ein Stück in die reflektierte Richtung. Um dies zu beachten, sollten wir die Rechnung noch etwas anpassen. 

Als Beispiel nehmen wir an, dass der Ball über $x=0$ hinaus fliegt.  Wenn wir also $c$ ausrechnen, gilt:  $c_x = c_x + v_x*\Delta t < 0 $.

Damit können wir ausrechnen, wie lange ($\Delta t_2$) unser Ball in die reflektierte Richtung geflogen ist: 

$abs(c_x)=\Delta t_2 * v_x$

also gilt:   
 
$\Delta t_2= \frac{abs(c_x)}{v_x}$. 

Für diese Zeit fliegt der Ball in die andere Richtung. Es gilt also 

$c_x = \Delta t_2 * (-v_x) = abs(c_x)$

<!-- BEGIN QUESTION -->

**Aufgabe:** Implementieren Sie die Funktion `timestep`, welche *einen* Zeitschritt für *einen Ball* simuliert. 

**Hinweis**:
- Die Entscheidungen, die Sie hier treffen, haben großen Einfluss auf die Performance. Es ist sehr wichtig, geeignete Kontroll- und Datenstrukturen zu wählen, um Overhead durch zu viele Objekte oder unnötige Kopien zu vermeiden.
- Wir haben hier eine schrittweise Simulation, dabei ist die Größe der Zeitschritte entscheidend, d.h. der Ball sollte in einem Zeitschritt nicht zu viel Strecke zurück gelegt haben. Daher sollte gelten: $\Delta_t \ll v$. Außerdem soll der Ball sich auch etwas bewegen können, daher muss $xmax,ymax \gg v$ sein.

_Points:_ 5

In [ ]:
def timestep(c, v, dt, g, xmax, ymax):
    """
    Simulate a timestep for a single ball.

    Parameters:
        c: ...
        v: ...
        dt: The duration of a single timestep.
        g: Gravitation
        xmax: Box limit in x direction
        ymax: Box limit in y direction
    """
    ...

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Aufgabe:** Animieren Sie die Bewegung des Balls. Der Ball sollte mit der Zeit an Höhe verlieren (durch die Schwerkraft).

Hinweis: Simulation und Visualisierung müssen getrennt sein (für die nächste Aufgabe).

_Points:_ 5

In [ ]:
import numpy as np

# Anfangsbedingungen (Vorschlag)
dt = 0.1
g = np.array([0, -9.81], dtype=np.float64)
xmax = 100
ymax = 100
steps = 100
# Startposition: x=500, y=500.
# ...
# Startgeschwindigkeit: v_x=20, v_y=0
# ...


...

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Aufgabe:** Ein Ball ist jedoch langweilig, daher wollen wir als nächstes mehrere Bälle simulieren.  
Schreiben Sie eine neue Funktion, `timestep_balls`, welche einen Array mit Koordinaten (shape=(N,2)) und einen Array mit Geschwindigkeiten (shape=(N,2)) übergeben bekommt und für alle Bälle einen Zeitschritt berechnet. 

Achten Sie darauf, dass der Code performant ist und verwenden Sie NumPys interne Vektorisierung mit der Slicing Syntax.  Für den Augenblick ignorieren wir, dass die Bälle zusammenstoßen können. 

Hinweis: 
- Das Array `c` sollte diese Form haben: 

$c = \left[\begin{array}{rr}                                 
x_0 & y_0 \\ 
x_1 & y_1 \\ 
x_2 & y_2 \\ 
\dots \\ 
x_{n-1} & y_{n-1} \\ 
\end{array}\right]$ 

- Das Array `v` sollte diese Form haben: 

$v = \left[\begin{array}{rr}                                 
v^x_0 & v^y_0 \\ 
v^x_1 & v^y_1 \\ 
v^x_2 & v^y_2 \\ 
\dots\\ 
v^x_{n-1} & v^y_{n-1} \\ 
\end{array}\right]$ 

_Points:_ 10

In [ ]:
def timestep_balls(c, v, dt, g, xmax, ymax):
    """
    Simulate a timestep for multiple balls.

    Parameters:
        c: ...
        v: ...
        dt: The duration of a single timestep.
        g: Gravitation
        xmax: Box limit in x direction
        ymax: Box limit in y direction
    """
    ...

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Aufgabe:** Animieren Sie die Bewegung der Bälle.

Tipp: Trennen Sie Simulation und Visualisierung.

_Points:_ 10

In [ ]:
import numpy as np

# Anfangsbedingungen (Vorschlag)
dt = 1
g = np.array([0, -9.81], dtype=np.float64)
xmax = 1000
ymax = 1000
steps = 250

# Anzahl Bälle
N = 10
rng = np.random.default_rng(seed=123)
# Startpositionen
c = xmax * rng.random((N, 2))
# Startgeschwindigkeiten
v = 10 * rng.random((N, 2))

...

<!-- END QUESTION -->

## Leistungsanalyse ohne Visualisierung

<!-- BEGIN QUESTION -->

**Aufgabe:** Analysieren Sie zunächst die Leistung der Ball-Simulation ohne Visualisierung mit Scalene!
Simulieren Sie dazu 100 Bälle für 100 Zeitschritte und verwenden Sie eine Abtastfrequenz von $0.0001$.

*Hinweis:* Wir haben die Initialisierungsfunktion aus dem Profiling entfernt, da sie ansonsten zu dominant wäre.
Wenn Sie den Code richtig optimiert haben, sollte die Funktion zu über 95% im nativen Modus ausgeführt werden.

_Points:_ 15

In [ ]:
%load_ext scalene

In [ ]:
# Anfangsbedingungen (Vorschlag)
dt = 1
g = np.array([0, -9.81], dtype=np.float64)
xmax = 1000
ymax = 1000
steps = 1000

# Anzahl Bälle
N = 100
rng = np.random.default_rng(seed=123)
# Startpositionen
c = xmax * rng.random((N, 2))
# Startgeschwindigkeiten
v = 10 * rng.random((N, 2))

In [ ]:
%%scalene --cpu-sampling-rate 0.0001
...

<!-- END QUESTION -->

## Leistungsanalyse mit Visualisierung

<!-- BEGIN QUESTION -->

**Aufgabe:** Die Visualisierung erfordert, dass die Ergebnisse nach jeder Iteration gespeichert werden. Analysieren Sie mit Hilfe von Scalene, wie sich das auf die Performance auswirkt. 

- Wie viel der gesamten Laufzeit kostet dieses Kopieren der Ergebnisse?
- Wie lässt sich das verbessern?

_Points:_ 15

_Type your answer here, replacing this text._

In [ ]:
%%scalene --cpu-sampling-rate 0.001
...

<!-- END QUESTION -->

## Einfaches Pendel (40 Punkte)

Die Bewegung eines einfachen Pendels kann man mit der Gleichung

$\theta^{''}(t) +\frac{b}{m}\theta^{'}(t) +\frac{g}{L}\theta(t) = 0$ 

beschrieben, wobei gilt:

- $\theta$ : Ausrichtungs-Wingel
- $b$ : Dämpfung (z.b. durch Luftwiderstand)
- $m$ : Masse des Pendel-Körpers
- $g$ : Erdbeschleunigung 
- $L$ : Länge des Pendels
- $\theta^{'}$: Winkel-Geschwindigkeit
- $\theta^{''}$: Winkel-Beschleunigung

Die Ausrichtungen x, y werden mit Hilfe von $\theta$ berechnet:

$ x = L * sin(\theta) $

$ y = -L * cos(\theta) $

Der Ursprung des Pendels liegt bei (0,0).

Das Pendel sei Anfangs zu $\frac{\pi}{3}$ ausgerichtet und hat eine Gechwindigkeit von 0. 

<!-- BEGIN QUESTION -->

**Aufgabe:** Berechnen Sie die Bewegung des Pendels für $t=[0,...,15]$ für 20000 Zeitschritte. Verwenden Sie die SciPy funktion `integrate.solve_ivp`.

_Points:_ 10

In [5]:
import numpy as np 
from scipy.integrate import solve_ivp 
from numba import njit

# Anfangsbedingungen
L = 1
m = 1
b = 0.5
g = 9.81
steps = 20000
T = 15
theta0 = np.pi/3 
omega0 = 0

@njit 
def pendulum(t, y): 
    theta, omega = y 
    theta_dot = omega 
    omega_dot = -(b/m)*omega - (g/L)*np.sin(theta) 
return [theta_dot, omega_dot]

#Initial conditions
y0 = [theta0, omega0]

#Time span
t_span = [0, 15]

#Time points
t = np.linspace(t_span[0], t_span[1], steps)

#Solve the differential equation
sol = solve_ivp(pendulum, t_span, y0, t_eval=t)

#Calculate x and y positions
x = Lnp.sin(sol.y[0]) 
y = -Lnp.cos(sol.y[0])

#Print the results
for i in range(len(t)): print(f"t = {t[i]}, x = {x[i]}, y = {y[i]}")

NameError: name 'np' is not defined

<!-- END QUESTION -->

**Aufgabe:** Die Funktion ` scipy.integrate.solve_ivp` bietet verschiedene Methoden zur Lösung an [[1]](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html). Vergleichen Sie die Performance für die Methoden, die keine weiteren Eingaben erwarten (RK45, RK23 und DOP853). 

_Points:_ 3

In [ ]:
rk45_timeit = ...
rk23_timeit = ...
dop853_timeit = ...

<!-- BEGIN QUESTION -->

**Aufgabe:** Stellen Sie die x- und y-Position des Pendels über die Zeit in einem Diagramm dar. 
Stellen Sie in einem zweiten Diagramm den Winkel über die Zeit dar.

Hinweis:
- Verwenden Sie die Lösung für die `RK45` Methode. 
- Verwenden Sie Liniendiagramme.

_Points:_ 10

In [ ]:
...

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Aufgabe:** Stellen Sie den Unterschied zwischen der 'RK45' Methode und der 'RK23' Methode dar. 

Erstellen Sie vier Liniendiagramme:
- x für beide Methoden über der Zeit
- y für beide Methoden über der Zeit
- Absoluter Unterschied von x zwischen beiden Methoden über der Zeit
- Absoluter Unterschied von y zwischen beiden Methoden über der Zeit

_Points:_ 10

In [ ]:
...

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Aufgabe:** Animieren Sie das Pendel, sodass es für mindestens eine Periodendauer pendelt.

Hinweis:
- Animieren Sie nicht alle Zeitschritte, sondern z.B. nur jeden 100. Zeitschritt.
- Damit die Animation schneller erstellt wird, können Sie z.B. die zweite Hälfte der Daten verwerfen.
- Tipp für "hvplot": aus `import holoviews as hv` ist [hv.Path() hilfreich](https://holoviews.org/reference/elements/bokeh/Path.html)

_Points:_ 7

In [ ]:
...

<!-- END QUESTION -->

Führen Sie alle Zellen im Notebook aus und speichern Sie es.  
Danach können Sie mit der folgenden Zelle eine HTML-Datei erstellen.  
Eine mögliche `UserWarning` können Sie ignorieren.  
Bitte geben Sie das Notebook als `.ipynb` und `.html` Datei ab.

In [ ]:
#!jupyter nbconvert --to html EA2.ipynb